In [1]:
import os
os.chdir(r"D:\Abhinav Tar Files\Snapshots") 

In [ ]:
import numpy as np
from ReadFile import Read
from CenterOfMass2 import CenterOfMass
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

G = 4.498e-6  # kpc^3 / (1e10 Msun * Myr^2)

def kinetic_energy(masses, velocities):
    """Compute total kinetic energy K = 0.5 * sum(m*v^2)"""
    v2 = np.sum(velocities**2, axis=1)
    return 0.5 * np.sum(masses * v2)

def potential_energy_barnes_hut(positions, masses, theta=0.7, show_progress=True):
    """
    Approximate gravitational potential energy using a Barnes–Hut-like approach.
    Complexity ~ O(n log n)
    show_progress: prints completion percentage
    """
    tree = cKDTree(positions)
    U = 0.0
    N = len(masses)

    progress_interval = max(N // 100, 1)  # Update every ~1% of particles

    for i, pos in enumerate(positions):
        # Query all particles
        dists, idxs = tree.query(pos, k=N, distance_upper_bound=np.inf)
        for r, j in zip(dists, idxs):
            if j <= i or r == 0 or np.isinf(r):
                continue
            U -= G * masses[i] * masses[j] / r

        # Print progress
        if show_progress and (i % progress_interval == 0 or i == N-1):
            percent = (i + 1) / N * 100
            print(f"Potential energy processing: {percent:.1f}%", end='\r')

    print()  # newline after progress
    return U

# --- MAIN LOOP ---
snapshots = np.arange(0, 802)
K_series = np.zeros(len(snapshots))
U_series = np.zeros(len(snapshots))
virial_ratio = np.zeros(len(snapshots))

count = 0
for idx, snap in enumerate(snapshots):
    mw_file = f"MW_{snap:03d}.txt"
    m31_file = f"M31_{snap:03d}.txt"

    MW = CenterOfMass(mw_file, 1)
    M31 = CenterOfMass(m31_file, 1)

    x = np.concatenate((MW.x, M31.x))
    y = np.concatenate((MW.y, M31.y))
    z = np.concatenate((MW.z, M31.z))
    vx = np.concatenate((MW.vx, M31.vx))
    vy = np.concatenate((MW.vy, M31.vy))
    vz = np.concatenate((MW.vz, M31.vz))
    m = np.concatenate((MW.m, M31.m))

    pos = np.vstack((x, y, z)).T
    vel = np.vstack((vx, vy, vz)).T

    # Center on MW COM
    xcom, ycom, zcom = MW.COMdefine(x, y, z, m)
    vxcom, vycom, vzcom = MW.COMdefine(vx, vy, vz, m)
    pos -= np.array([xcom, ycom, zcom])
    vel -= np.array([vxcom, vycom, vzcom])

    # Compute energies
    K_series[idx] = kinetic_energy(m, vel)
    print(f"Processing snapshot {snap}: computing potential energy...")
    U_series[idx] = potential_energy_barnes_hut(pos, m, show_progress=True)
    virial_ratio[idx] = 2 * K_series[idx] / np.abs(U_series[idx])

    count += 1
    print(f"Processed {count}/{len(snapshots)} snapshots (last = {snap})\n")

# --- PLOT RESULTS ---
plt.figure(figsize=(10,6))
plt.plot(snapshots, K_series, label="Kinetic Energy K")
plt.plot(snapshots, np.abs(U_series), label="|Potential Energy U|")
plt.plot(snapshots, virial_ratio, label="Virial Ratio 2K/|U|")
plt.xlabel("Snapshot Number")
plt.ylabel("Energy / Ratio (in 1e10 Msun, kpc, Myr units)")
plt.title("Energy Evolution of MW+M31 Halo")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("halo_energy_evolution.png", dpi=300)
plt.show()


Processing snapshot 0: computing potential energy...
Potential energy processing: 0.0%